In [1]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
# rayfile = "/home/anuran/repos/Vulcan/bin/rayDump.rays"
rayfile = "/bbb3_glass_eval_111716.rays"
# rayfile = "/bbb3_glass_eval_fast_112016.rays"
raypath = os.getcwd()+rayfile


In [3]:
# load rays
rays_struct = np.dtype(
    [
        ('sx', np.float32),
        ('sy', np.float32),
        ('ex', np.float32),
        ('ey', np.float32),
        ('r' , np.float32),
        ('th', np.float32),
        ('i',  np.uint16),
        ('pad',np.uint16),
        ('idx',np.int32),
        ('ts', np.float64), #remove for old ray file formats
      ])

# rays = np.memmap(rayfile,rays_struct, 'r',1)
rays0 = np.fromfile(raypath, rays_struct)
rays0 = rays0[:int(len(rays0)//1081*1081)]

In [4]:
# split rays by sensor THIS IS A HACK RIGHT NOW!
# .... actually, don't bother

In [5]:
# #Debug Cell
# %matplotlib widget
# # quickly plot the bot path and rays
# substart=int(8.0e6)
# subend=int(11.0e6)
# substart=int(substart//1081)*1081
# subend=int(subend//1081)*1081
# rayss = rays0[substart:subend]
# rayss = rayss[np.abs(rayss['r'])<50]
# plt.plot(rayss['ex'][::],rayss['ey'][::1],',')
# plt.plot(rayss['sx'][::997],rayss['sy'][::997],',')
# del rayss

# print("Total starting rays:", len(rays0))

In [6]:
# #HACK use only one pass for algo debug
# rays0 =  rays0[substart:subend]
# rays0 = rays0[:int(len(rays0)//1081*1081)]

# # HACK: pull out the rays for Fig 1
# ok=(rays0['ts']>rays0['ts'][0]+208) & (rays0['ts']<rays0['ts'][0]+225)
# rays0=rays0[ok]
# rays0 = rays0[:int(len(rays0)//1081*1081)]

# Hyperparameter Setting Instructions
1. CELLS_PER_M should indicate a cell width less than the max displacement of the robot between scans. 
   1. Otherwise there may be gaps between adjacent rays in the same direction.
2. Theta width should be >= laser angular spacing, and <= min peak half-width at worst laser plane tilt
   1. If you can't satisfy both, mount your laser better, stiffen your suspension, or get a blurrier (higher divergence) laser!
3. Max range should be set so max_turn_rate * max_range < cell_width
4. Currently, the max distance is hardcoded at 200 cells


In [7]:
## Find the max extent of the map and transforms

keep=(rays0['r']<10000)*(rays0['r']<10000)
rays=rays0[keep==1]

## Figure out rasterization
# Floor of float coords is integer coords
# Images are y flipped

def homp(T,pts):
    tmppts = pts @ T[:-1,:-1].T + T[:-1,-1]
    denom = pts @ T[-1:,:-1].T + T[-1,-1]
    return tmppts/denom


CELLS_PER_M = 20.0
print("Cell width:", 1/CELLS_PER_M)

formatter = {'float':lambda x:np.format_float_positional(x,precision=2,fractional=True,trim='-',pad_left=8,pad_right=2)}

def get_T_px_f_with_extent(rays, debug=False):
    # The corners in world space
    f_corns = np.array(
        [[np.min(rays['ex']),np.min(rays['ey'])],
         [np.max(rays['ex']),np.max(rays['ey'])]])
    T_pxc_f = np.array(
        [[1*CELLS_PER_M, 0             , 0],
         [0            , -1*CELLS_PER_M, 0],
         [0            , 0             , 1]])
    pxc_corns=homp(T_pxc_f,f_corns)
    pxc_topleft = np.array([  np.floor(np.min(pxc_corns[:,0])), np.floor(np.min(pxc_corns[:,1]))  ]) #inclusive
    pxc_btmright= np.array([  np.floor(np.max(pxc_corns[:,0])), np.floor(np.max(pxc_corns[:,1]))  ]) + 1 #exclusive
    pxc_extent = pxc_btmright-pxc_topleft

    T_px_f = T_pxc_f.copy()
    T_px_f[:-1,-1] = -pxc_topleft

    px_corns=homp(T_px_f,f_corns)
    px_topleft = np.array([  np.floor(np.min(px_corns[:,0])), np.floor(np.min(px_corns[:,1]))  ]) #inclusive
    px_btmright= np.array([  np.floor(np.max(px_corns[:,0])), np.floor(np.max(px_corns[:,1]))  ]) + 1 #exclusive
    px_extent = px_btmright-px_topleft

    assert (np.all(pxc_extent == px_extent))

    if debug:
        print("fcorns\n",f_corns)

        for var in ['pxc_corns',
                    'pxc_topleft',
                    'pxc_btmright',
                    'pxc_extent',
                    'px_corns',
                    'px_topleft',
                    'px_btmright',
                    'px_extent',
                    ]:
            print(var+'\n', eval(var))
    return T_px_f, px_extent.astype('int64')

keep_for_window=(rays0['r']>0)*(rays0['r']<10)
with np.printoptions(formatter=formatter):
    T_px_f, px_extent = get_T_px_f_with_extent(rays[keep_for_window], debug=True)

Cell width: 0.05
fcorns
 [[      28.71       12.34]
 [      70.15       76.33]]
pxc_corns
 [[     574.11     -246.81]
 [    1402.93    -1526.6 ]]
pxc_topleft
 [     574       -1527   ]
pxc_btmright
 [    1403        -246   ]
pxc_extent
 [     829        1281   ]
px_corns
 [[       0.11     1280.19]
 [     828.93        0.4 ]]
px_topleft
 [       0           0   ]
px_btmright
 [     829        1281   ]
px_extent
 [     829        1281   ]


In [8]:
%matplotlib widget
plt.plot(rays0['ex'],rays0['ey'],'b,')
# # # If you need to edit the rays, but keep extents, do it here
# # rays0 = rays0[::53]
print(len(rays0))
# rays0 = rays0[rays0['r']<10]
# rays0 = rays0[rays0['ex']>50 ]
# rays0 = rays0[rays0['ex']<55 ]
# rays0 = rays0[rays0['ey']<23 ]
print(len(rays0))

# rays0 = rays0[rays0['ts']< 1479396535.3080368]
rays = rays0.copy()

print(len(rays0))
# rays = rays0.copy()
plt.plot(rays0['ex'],rays0['ey'],'k,')
plt.plot(rays0['sx'],rays0['sy'],'c,')
print(np.min(rays0['ts']),np.max(rays0['ts']))

# convert to [start_cell_x,start_cell_y, d_cell, th] format
f_spt = np.vstack((rays['sx'].ravel(),rays['sy'].ravel())).T.copy()
f_ept = np.vstack((rays['ex'].ravel(),rays['ey'].ravel())).T.copy()
px_spt = homp(T_px_f, f_spt)
px_ept = homp(T_px_f, f_ept)
px_d = np.sqrt(np.sum((px_ept-px_spt)**2,axis=1))
px_th = np.mod(np.arctan2((px_ept-px_spt)[:,1], (px_ept-px_spt)[:,0]),2*np.pi)
def get_raster_coords(rays, T_px_f):
    # implicit inputs: T_px_f
    f_ept = np.vstack((rays['ex'].ravel(),rays['ey'].ravel())).T.copy()
    px_coords = homp(T_px_f, f_ept)
    px_coords = px_coords.reshape(rays['ex'].shape+(2,))
    return px_coords

th_spacing = 2*np.pi/360/4

outrays = np.vstack([px_spt.T,px_d,px_th/th_spacing])
notref = np.ones_like(outrays[0], dtype=bool)


# plt.plot(px_ept[:,0],px_ept[:,1],'b,')
# plt.plot(px_ept[10000:101081,0],px_ept[10000:101081,1],'r,')




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

23674981
23674981
23674981
1479396473.584265 1479396811.672591


In [9]:
%matplotlib widget
import numpy as np
th_spacing = 2*np.pi/360/4 # 1/4 degree
th_bins = int(np.round(2*np.pi / th_spacing)) 
W, H = 800,1200

w_HIT = 1
w_MISS = 4 # Reduce to 1 if known no motion
bias = 0
wide_threshold = 30*np.pi/180 / th_spacing
epsl = 1.

# Derived constants
# REF_BLOCK_THRESH = w_MISS/(w_MISS+w_HIT) # the theoretical way. Fraction of hits to total that means we have a reflective voxel
REF_BLOCK_THRESH = 0.3 # where does 0.3 come from? Don't know. Tested it somehow


HIT = np.zeros((W,H,th_bins),'int16')
MISS = np.zeros((W,H,th_bins),'int16')
RFM = np.zeros((W,H,th_bins),'int8') # -1=TRANSPARENT, 0=UNK, 1=REFLECT # but some code just uses 0=TRANS, 1=REFLECT
RFM_mr = np.zeros((W,H,th_bins),'int8')
WIDE = np.zeros((W,H),'bool')
NARROW = np.zeros((W,H),'bool')
prerender = np.zeros((W,H),'bool')
refl_cache = np.zeros((W,H),'int16')
trans_cache = np.zeros((W,H),'int16')
OCC = np.zeros((W,H),'int8')

vox_refl = np.zeros((W,H,th_bins),'float32')  
CLASSIFIED_RFM = np.zeros((W,H,th_bins),'bool')  
countvis = np.zeros((W,H),'uint16')
counttrans = np.zeros((W,H),'uint16')
countratio = np.zeros((W,H),'float32')

FDIST = np.full((W,H,th_bins), 32767, 'int16')

flood_frontier = []
next_flood_frontier = []

def RFM_update_cell(x,y,th):
    # updates the RFM cell and the cached values 
    old_val = RFM[x,y,th]
    new_val = np.sign(w_HIT*HIT[x,y,th]-w_MISS*MISS[x,y,th])
    RFM[x,y,th] = new_val
    if old_val == -1:
        trans_cache[x,y] -=1
    if old_val == 1:
        refl_cache[x,y] -=1
    if new_val == -1:
        trans_cache[x,y] +=1
    if new_val == 1:
        refl_cache[x,y] +=1
        
    if new_val != old_val:
        flood_frontier.append((x,y,th))
        

def quantize(coords):
    return np.floor(coords).astype('int')
    
def quantize4(coords): # 4 pt xy antialiasing samples
    coords = np.array(coords)
    coords4 = (coords + (-0.5,-0.5,0.0), coords + (-0.5,0.5,0.0), 
               coords + (0.5,-0.5,0.0), coords + (0.5,0.5,0.0))
    return np.floor(coords4).astype('int')

tmp=np.hstack((np.arange(27,dtype=np.int32)[0:13],np.arange(27,dtype=np.int32)[14:]))
NEIGHBORS = np.array((   tmp//9-1,
                         tmp//3%3-1,
                         tmp%3-1,)).T

cos=np.cos
sin=np.sin
def accumulate(rays):
    # prerender lets us stop rendering a ray that would go through a HIT
    prerender[:,:]=0
    for ray in rays:
        # ray has a starting point, distance before it returned, and direction
        [x_start, y_start, d, th] = ray
        [slope_x, slope_y] = [cos(th*th_spacing), sin(th*th_spacing)]
    
        xyth = quantize4((x_start + d*slope_x, y_start + d*slope_y, th))
        prerender[xyth[:,0],xyth[:,1]] = True
    
    for ray in rays:
        # ray has a starting point, distance before it returned, and direction
        [x_start, y_start, d, th] = ray
        [slope_x, slope_y] = [cos(th*th_spacing), sin(th*th_spacing)]
    
        for r in range(0,int(d-epsl)+1):
            xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
            if not prerender[xyth[0],xyth[1]]:
                MISS[xyth[0],xyth[1],xyth[2]]+=1
                RFM_update_cell(xyth[0],xyth[1],xyth[2])
            else:
                break
        xyth = quantize4((x_start + d*slope_x, y_start + d*slope_y, th))
        HIT[xyth[:,0],xyth[:,1],xyth[:,2]]+=1
        for xyth_sample in xyth:
            RFM_update_cell(xyth_sample[0], xyth_sample[1], xyth_sample[2])

    # Do a step of floodfill
    
    # First mark any changed WIDE state to investigate
    if 1:
#         refl_cache[5,:]=700 #HACKKKKKKK
#         RFM[5,:,:]=1
        
        old_WIDE = WIDE.copy()
        classify_wide_narrow()
        change_cols = old_WIDE!=WIDE


        # get indicies of RFM[change_cols,:]
        flattmp = np.ravel_multi_index(np.nonzero(change_cols)+(np.arange(1440,dtype=np.int32)[:,np.newaxis],),
                                       RFM.shape).T.ravel()
#         flattmp = flattmp[RFM.ravel()[flattmp]==1]
        indices = np.unravel_index(flattmp,RFM.shape)
        ##Very expensive check of correctness
        if 0:
            it = np.indices(RFM.shape)
            assert (np.all(it[0].ravel()[flattmp]==it[0][change_cols,:].ravel()))
            assert(np.all(RFM[indices]==RFM[change_cols,:].ravel()))
        indices = np.array(indices).T
        print(indices)
        print("before cols ups:", len(flood_frontier))
        #only need to worry about setting distance properly on things that are occupied
        flood_frontier.append(indices)
        flood_frontier_a = np.vstack(flood_frontier)
        print("totaled",len(flood_frontier_a))
        flood_frontier_a = np.unique(flood_frontier_a, axis = 0)
        print("totaled u",len(flood_frontier_a))
    # Now scuttle 
    
    flood_frontier_a = (NEIGHBORS[:,np.newaxis,:]+flood_frontier_a).reshape(-1,3)
    print("totaled nabs",len(flood_frontier_a))
#     flood_frontier.update([tuple(idx) for idx in indices])
#     print(len(flood_frontier))
#     print(flood_frontier)
#     print(np.ravel_multi_index([
#                                 np.array([1,2,3])[:,np.newaxis],
#                                 np.array([0,0,0,])[:,np.newaxis],
#                                 np.array([0,0,0])[np.newaxis:]
#                                 ], RFM.shape))
    plt.imshow(np.sum(RFM>0,axis=2))
    
#     change_cols = old_WIDE != WIDE # columns where wide visibility changed
#     RFM[changecols,:]>0
#     # 
#     FDIST[WIDE[:,:],:] = (RFM[WIDE[:,:],:]<=0)*65535
    
#     next_flood_frontier.clear()
    
#     test = []
#     for cell in flood_frontier:
#         if RFM[cell]!=1:
#             FDIST[cell]=65535
            
#         next
#         test.append(NEIGHBORS+cell) 
#     test=np.vstack(test).T.copy()
#     print(cell)
#     print("to check")
#     print(test)
#     FDIST[np.ravel_multi_index(test, RFM.shape, mode='clip')
print( outrays[:1000])   
# accumulate(outrays.T[:1000])
        

[[ 205.69459534  205.69459534  205.69459534 ...  211.94543457
   211.94543457  211.94543457]
 [ 150.69857788  150.69857788  150.69857788 ...  144.15454102
   144.15454102  144.15454102]
 [1581.81201997 1581.81201997 1581.81201997 ... 1590.58850416
  1590.58850416 1590.58850416]
 [ 478.12871511  478.12871511  478.12871511 ...  478.44759237
   478.44759237  478.44759237]]


In [10]:

        
def classify_wide_narrow():
#     global WIDE
    WIDE[:,:] = refl_cache > wide_threshold
#     WIDE &= refl_cache/trans_cache > wide_min_ratio # Optional: make sure the ratio is good
    NARROW[:,:] = (refl_cache>0) & (WIDE==False)

def remove_motion():
    RFM_mr[NARROW,:] = RFM[NARROW,:]-3 # map [-1,0,1]->[-4,-3,-2]
    RFM_mr[WIDE,:] = RFM[WIDE,:]
    stack = []
    seed_locs=np.argwhere(WIDE)
    for x,y in seed_locs:
        if (  NARROW[x-1,y-1] or NARROW[x-1,y] or NARROW[x-1,y+1] or
              NARROW[x,y-1] or NARROW[x,y+1] or
              NARROW[x+1,y-1] or NARROW[x+1,y] or NARROW[x+1,y+1]  ):
            thetas = RFM[x,y,:]
            for th in thetas:
                stack.append((x,y,th))
    while(stack):
        x,y,th = stack.pop
        t2d_neighbors = ((x-1,y-1),(x-1,y),(x-1,y+1),
                        (x,y-1),  (x,y)  ,(x,y+1),
                        (x+1,y-1),(x+1,y),(x+1,y+1))
        for x,y in t2d_neighbors:
            if not NARROW(x,y): continue
            for th in (th-1, th, th+1):
                if RFM_mr[x,y,th] == -2: # Unvisited, NARROW, REFL
                    RFM_mr[x,y,th] = 4
                    stack.append((x,y,th))
    # codes: -4:NAR,TRANS -3:NAR, UNK -2:NAR,DYN 
    #        -1:WIDE,TRANS 0:WIDE,UNK  1:WIDE,STATIC
    #         4:NAR,CON
    RFM_mr[NARROW,:] //= 4 # remap -4,-3,-2,4 -> -1,0,0,1; deleting motion
    # Classify occupancy: -2=FREE, -1=DYNAMIC, 0=UNK, 1=STATIC
    OCC[:,:] = 0 
    OCC[trans_cache>0] = -2 # seen through defaults FREE
    OCC[NARROW] = -1 # narrow defaults DYNAMIC
    OCC[WIDE] = 1
    # The following line is an optimization of
    # OCC[np.any(RFM_mr==1,axis=1)] = 1
    OCC[NARROW] += np.any(RFM_mr[NARROW,:]==1,axis=1)*2
    OCC[NARROW & (transcache==0)] = 1 # Default uncontested cells to STATIC

def classify_reflections(rays):
    is_reflection = []
    epsl = 1.999

    for ray in rays:
        [x_start, y_start, d, th] = ray
        [slope_x, slope_y] = [cos(th*th_spacing), sin(th*th_spacing)]
    
        for r in range(0,int(d-epsl)+1):
            xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
            if OCC[xyth[0], xyth[1]]:
                is_reflection.append(True)
                break
        else:
            is_reflection.append(False)
    return is_reflection

def process_scan(rays):
    # This is the do-everything main function
    accumulate(rays)
    classify_wide_narrow()
    remove_motion()
    # Now the RFM estimate without motion removal is in RFM
    # and the version with the motion removed is in RFM_mr.
    # OCC holds the estimated occupancy classification
    
    # classify_rays() can be used to check rays for reflection in
    # another thread, and build a new RFM without them

# Basic RFM
The following calculates the basic RFM, accumulating HITs and MISSes. \
It does not account for reflections, and doesn't link up objects (we do that later)

Typically runs ~80 scans/second

In [11]:
loclip = np.array([0,0,0]).reshape(3,1)
hiclip = (np.array(HIT.shape)-1).reshape(3,1)
def quantize(coords):# 3xN
    return np.clip(np.floor(coords),loclip,hiclip).astype('int')

def quantize4(coords): # 4 pt xy antialiasing samples # 3xN
    coords = np.array(coords).T  # Nx3
    aa = np.array(((-0.5,-0.5,0.0), (-0.5,0.5,0.0), 
                  ( 0.5,-0.5,0.0), ( 0.5,0.5,0.0) )) # 4x3
    coords4 = aa + coords[:,np.newaxis,:] # N,4,3
    coords4= coords4.reshape(-1,3).T
    return np.floor(np.clip(coords4,loclip,hiclip)).astype('int')

import pandas as pd
HMT = 1/(1/REF_BLOCK_THRESH-1)
def RFM_update_cell(x,y,th):
#     indices = np.ravel_multi_index((x,y,th),RFM.shape)
#     x,y,th = np.unravel_index(pd.unique(indices),RFM.shape)
    
    old_val = RFM[x,y,th]
    
    new_val = HMT < HIT[x,y,th]/(MISS[x,y,th]+np.float32(0.1))
    RFM[x,y,th] = new_val
    
    rcxy0 = refl_cache[x,y]
    wide0 = rcxy0 > wide_threshold
    np.add.at(refl_cache, (x,y), new_val-old_val)
    
    
    #anything that went from WIDE to NARROW needs update
    rcxy = refl_cache[x,y]
    wide = rcxy > wide_threshold
    to_wipe = (wide0)&(wide==0)
    FDIST[x[to_wipe],y[to_wipe],:] = 32767
    #debug wiping
    wipe_count = np.sum(to_wipe)
    if wipe_count:
        print("wiped count:", np.sum(to_wipe))
        
    #anything that went from NARROW to WIDE needs to be seeded
    to_seed = (wide)#&(wide0==0) #why does it break if I uncomment?
    FDIST[x[to_seed],y[to_seed],:] *= (RFM[x[to_seed],y[to_seed],:]==0)
    
    #update wide/narrow
    WIDE[x,y] = wide
    narrow = (rcxy>0) & (wide==False)
    NARROW[x,y] = narrow
#     assert HMT < HIT[x,y,th]/(MISS[x,y,th]+np.float32(0.1))
    
#     frontier = np.nonzero(new_val!=old_val)[0]
#     if len(frontier):
#         print("cells added",np.sum(new_val>old_val))
#         print("cells removed",np.sum(old_val>new_val))
# #         print("changed in accumulate", frontier.shape)
#     upped = x[frontier],y[frontier],th[frontier]
#     flood_frontier.append((x[frontier],y[frontier],th[frontier]))


    
# Define the accumulate operator, that renders rays
def accumulate(rays, notref):
    flood_frontier.clear()
    
    # prerender lets us stop rendering a ray that would go through a HIT
    prerender[:,:]=0
#     for ray in rays:
    # ray has a starting point, distance before it returned, and direction
    [x_start, y_start, d, th] = rays
    [slope_x, slope_y] = [np.cos(th*th_spacing), np.sin(th*th_spacing)]

    xyth = quantize4((x_start[notref] + d[notref]*slope_x[notref], y_start[notref] + d[notref]*slope_y[notref], th[notref]))
    prerender[xyth[0],xyth[1]]=1
    
#     for ray in rays:
    # ray has a starting point, distance before it returned, and direction
    [x_start, y_start, d, th] = rays
    [slope_x, slope_y] = [np.cos(th*th_spacing), np.sin(th*th_spacing)]
    
    
    
    live=np.ones_like(d)
    r=np.full_like(d,0)
    for r0 in range(0,200):
#         print(r0, end=' ')
        r[:]=r0
        live[r0>=d-epsl]=0
#         print(np.sum(live))
        r[live==0]=100000
        xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
#         if r0>1
        live*=(prerender[xyth[0],xyth[1]]==0)
        MISS[xyth[0],xyth[1],xyth[2]]+=1
#         RFM_update_cell(xyth[0],xyth[1],xyth[2]) # for speed, only update on hits
    xyth = quantize((x_start[notref] + d[notref]*slope_x[notref], y_start[notref] + d[notref]*slope_y[notref], th[notref]))
    HIT[xyth[0],xyth[1],xyth[2]]+=1
    RFM_update_cell(xyth[0], xyth[1], xyth[2])
#     print(np.hstack(flood_frontier).shape)

In [12]:
# The basic RFM doer. 
def doRFM():
    print("Building RFM")
    MISS[...] = 0
    HIT[...] = 0
    arays = outrays.copy()

    ls=0
    keep=(rays0['r']<10)*(rays0['r']>0)
    CHUNK_SIZE = 1081
    for s in tqdm(range(CHUNK_SIZE,arays.shape[1]+1,CHUNK_SIZE)):
            tmprays=arays[:,ls:s][:,keep[ls:s]]
            nrtemp=notref[ls:s][keep[ls:s]]
            accumulate(tmprays, nrtemp)
            ls=s
            if (ls//CHUNK_SIZE)%100==0:
                floodstep()
    plt.imshow(np.sum(MISS,axis=2))
    assert np.max(MISS)>0
    vox_refl[...] = HIT/(np.float32(0.1)+HIT+MISS) # classify 
    CLASSIFIED_RFM[...] = vox_refl > REF_BLOCK_THRESH
    countvis[...] = np.sum(CLASSIFIED_RFM, axis=2).astype(np.uint16)
    counttrans[...] = np.sum((CLASSIFIED_RFM==0)&(MISS>0), axis=2).astype(np.uint16)
    countratio[...] = np.float32(1.0*countvis/(countvis+counttrans+0.0000000001))
    refl_cache[...] = (countratio > 0.5) | (countvis > 12)  # More reflective than not or a suspiciously wide range of sightings

In [13]:
# Utility that unions two sets of labelled data. Used to turn normal flood fill into circular floodfill around theta.
def unify_labels(rfmcomps, rfmcomps2):
    # unify labels is mostly a wrapper around fuse, but with the optimization that we only try to 
    # fuse nonzero labels, since we know they're the same
    def fuse(A,B):
        # Assumes A and B are label matricies with different labels and we need to fuse them
        # Any repeated label between the two corresponds
        # Assumes that the labels go from 0 to max(max(A),max(B))
        # Also assumes all labels in A are <= to the ones in B
        assert(np.all(A<=B))
  
        lookup = np.arange(np.maximum(np.max(A),np.max(B))+1, dtype=np.int32)
        np.minimum.at(lookup,B,A) # Bs now lookup the lowest A they connect to, As loopback

        B=lookup[B] #Now all Bs are <= A

        np.minimum.at(lookup,A,B)
        #As lookup the smallest B they connect to, Bs lookup the smallest A or better
        B=lookup[lookup[B]] 

        extr=(A!=B) 
        remA=A[extr]
        remB=B[extr]
        if len(remA)==0:
            
            return B
        recur = fuse(remB,remA)
        B[extr] = recur
        return B
    
    nz=fuse(rfmcomps[rfmcomps!=0].ravel(),rfmcomps2[rfmcomps2!=0].ravel())
    rfmcomps[rfmcomps!=0]=nz
    return rfmcomps

# fuse(np.array([1,2,2,3,3,4,4]),np.array([5,6,7,7,8,8,9]))

# Classify Reflection Rays
Mark the depths where rays try to go through another object. 
PEN_DEPTH controls this. We set to 30cm to deal with the stupid 20cm short detections that seem to plague our laser.

The basic idea is that all rays render MISSes, but only rays that travel less than PEN_DEPTH from the surface can render HITs

In [14]:
# classify reflectionness of a ray and clip to a given penetration depth
PEN_DEPTH = np.ceil(0.101*CELLS_PER_M)
def classify(rays, pen_depth=None):
    rays=rays.copy()
    if pen_depth is None:
        pen_depth = PEN_DEPTH
    HACK_FACTOR = 2 # ensure that detected reflections still count as seen through for this distance
#     for ray in rays:
    # ray has a starting point, distance before it returned, and direction
    [x_start, y_start, d, th] = rays
    [slope_x, slope_y] = [np.cos(th*th_spacing), np.sin(th*th_spacing)]
    
    
    notref=np.ones_like(d,dtype=bool)
    r=np.full_like(d,0)
    for r0 in tqdm(range(0,200)):
#         print(r0,end=', ')
        r[:]=r0
        xyth = quantize((x_start + r*slope_x, y_start + r*slope_y, th))
        
#         if r0>1
        stopped=(refl_cache[xyth[0],xyth[1]]!=0)
        notrefish = r0 > d[stopped]-pen_depth
        notref[stopped] *= notrefish
#         print(r0+pen_depth)
       
        d[stopped]=np.minimum(d[stopped],r0+pen_depth+HACK_FACTOR) 
        
    rays = np.vstack([x_start, y_start, d, th])
    return rays, notref


# The returned rays now have distances reflecting how far to render them. 
# If the distance is less than the original, we want to not render the HIT when we rerender
# "notref" keeps track of which rays should render their endpoint

# This operation is ~1/3 the work of the normal RFM render, but I haven't optimized it, so it 
# runs much slower when the ray list is too big for the cache

In [26]:
%matplotlib widget
from timeit import timeit
import time
def floodstep():
    # This is a very basic version of floodstep.
    # It simply takes all NARROW points and looks through their 26 neighbors 
    # to find lowest distance to the seed
    tic = time.perf_counter()
#     assert np.all(np.sum(RFM,axis=2)==refl_cache)
    oldwide=WIDE.copy()
    classify_wide_narrow()
    assert np.all(oldwide==WIDE)
    toc = time.perf_counter(); print("classify_wide_narrow",toc-tic); tic=toc
    assert(not np.any(RFM[WIDE,:]&(FDIST[WIDE,:]!=0)))
    
#     #Set widely visible points to have dist=0
#     xy = np.nonzero(WIDE)
#     towers = RFM[xy[0],xy[1],:]
#     i2, ith = np.nonzero(towers)
#     FDIST[WIDE,:] = np.where
    
    xy = np.nonzero(NARROW)
    print("NARROW FOUND:",len(xy[0]))
    towers = RFM[xy[0],xy[1],:]
    center_samp2didx, center_thidx = np.nonzero(towers)
    minvals = np.full(center_thidx.shape, 32767, np.int16)
    toc = time.perf_counter(); print("extract 2d locs",toc-tic); tic=toc
    
    # At this stage, the points that we want to flood to are 
    #    xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx
    for xoff in [-1,0,1]:
        for yoff in [-1,0,1]:
            tic2 = time.perf_counter()
            sampx, sampy = xy[0]+xoff,xy[1]+yoff
            flat_samp_2d = np.ravel_multi_index((sampx,sampy),refl_cache.shape,'clip') #2d locs
            # Which 2d samples are worth trying(have any ref cells)?
            flat_samp_2d_is_ok = refl_cache.ravel()[flat_samp_2d]>0 #dims: (#2d inputs,) 
            assert np.all(refl_cache[xy[0][center_samp2didx],xy[1][center_samp2didx]]>0)
            #Of the 3d samples we would want to try, which are ok?
            samp3d_is_ok = flat_samp_2d_is_ok[center_samp2didx] # we want to sample these 2ds. Which to keep?
            samp3d_is_ok = np.nonzero(samp3d_is_ok)
            
            #remove samples not ok from 2d info
            idx2d_for_3dsamples = center_samp2didx[samp3d_is_ok]
#             print("idx2d_for_3dsamples.shape", idx2d_for_3dsamples.shape)
            center_thidx_for_3dsamples = center_thidx[samp3d_is_ok]
            # centers to flood to will be:
            #    xy[0][idx2d_for_3dsamples],xy[1][idx2d_for_3dsamples],center_thidx_for_3dsamples
            # and samples will be:
            #    sampx[idx2d_for_3dsamples],sampy[idx2d_for_3dsamples],center_thidx_for_3dsamples + <some offset>
            
            # Now we offset in theta to get the final sample points
            for thoff in [-1,0,1]:
                tic3 = time.perf_counter()
                if xoff==0 and yoff==0 and thoff==0:
                    continue
                samp3d_idx = np.ravel_multi_index((sampx[idx2d_for_3dsamples],
                                                      sampy[idx2d_for_3dsamples],
                                                      center_thidx_for_3dsamples + thoff
                                                     ),
                                                     RFM.shape,
                                                     'clip')
                np.minimum.at(minvals, samp3d_is_ok, FDIST.ravel()[samp3d_idx])
#                 toc = time.perf_counter(); print("th loop",toc-tic3)
#             toc = time.perf_counter(); print("2d loop",toc-tic2)
    toc = time.perf_counter(); print("flooding",toc-tic); tic=toc
#     assert not np.any(FDIST[xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx]==0)
    oldval = FDIST[xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx]
    minvals[minvals==oldval]=32767 # if value is rising, we probably just lost our support
    newval = minvals+(minvals<32767)
    
    changed = FDIST[xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx]!=newval
    print("change count", np.sum(changed))
    FDIST[xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx]=newval
    print(np.vstack((xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx, newval))[:,changed])
    toc = time.perf_counter(); print("writeback",toc-tic); tic=toc
#     fdt=FDIST
#     t=timeit('fdt[xy[0][center_samp2didx],xy[1][center_samp2didx],center_thidx]=minvals',globals=locals(),number=1)/1
#     print("writeback time:", t)
    return minvals

FDIST[FDIST<0]=32767

t=timeit('classify_wide_narrow()',globals=globals(),number=1)/1
print("classify_wide_narrow time:", t)
FDIST[:,:,:]=32767
t=timeit('FDIST[WIDE,:]=np.where(RFM[WIDE,:]==1,0,32767)',globals=globals(),number=1)/1
print("seeding time:", t)
t=timeit('floodstep()',globals=globals(),number=1)/1
print("floodstep time:", t)
print("# wide locs", np.sum(WIDE))
print("# wide cells", np.sum(RFM[WIDE,:]==1))

for i in range(10):
    t=timeit('floodstep()',globals=globals(),number=1)/1
    print("floodstep time:", t)
t=timeit('floodstep()',globals=globals(),number=1)/1
print("floodstep time:", t)
# minvals=floodstep()
# minvals[minvals>50]=50
# plt.hist(minvals,range(51))
# print (floodstep()[1])

classify_wide_narrow time: 0.0021739050062024035
seeding time: 0.0309228650003206
classify_wide_narrow 0.004104123996512499
NARROW FOUND: 65171
extract 2d locs 0.3180005140020512
flooding 2.2166709209996043
change count 462409
[[ 153  153  153 ...  738  738  738]
 [ 822  822  822 ...  825  825  825]
 [ 565  566  567 ... 1437 1438 1439]
 [   1    1    1 ...    1    1    1]]
writeback 0.05445597200014163
floodstep time: 2.5934577980005997
# wide locs 8284
# wide cells 2765792
classify_wide_narrow 0.003010995002114214
NARROW FOUND: 65171
extract 2d locs 0.301041668994003
flooding 2.187928497005487
change count 86652
[[ 152  152  152 ...  739  739  739]
 [ 822  822  822 ...  826  826  826]
 [ 565  567  568 ... 1341 1347 1439]
 [   2    2    2 ...    2    2    2]]
writeback 0.050693846998910885
floodstep time: 2.5428919910045806
classify_wide_narrow 0.003156166996632237
NARROW FOUND: 65171
extract 2d locs 0.30221955100569176
flooding 2.190034330000344
change count 51903
[[ 151  151  151 ...

In [17]:
keeprays=(rays0['r']<10)*(rays0['r']>0) # needed in ablations

In [18]:
doRFM()

Building RFM


  0%|          | 106/21901 [00:01<05:09, 70.38it/s]

classify_wide_narrow 0.002321910003956873
NARROW FOUND: 1295
extract 2d locs 0.008355778998520691
flooding 0.008702150997123681
change count 0
[]
writeback 0.0007062660006340593


  1%|          | 207/21901 [00:02<04:53, 73.96it/s]

classify_wide_narrow 0.002150633998098783
NARROW FOUND: 1330
extract 2d locs 0.007836136006517336
flooding 0.00904885699856095
change count 0
[]
writeback 0.0006964929998503067


...

100%|█████████▉| 21899/21901 [10:54<00:00, 65.46it/s]

classify_wide_narrow 0.00238574299874017
NARROW FOUND: 70922
extract 2d locs 0.3480932969978312


100%|██████████| 21901/21901 [10:56<00:00, 33.35it/s]

flooding 2.1689423019997776
change count 3247
[[  59   59   59 ...  738  738  738]
 [1123 1125 1125 ...  771  773  778]
 [ 681  681  682 ... 1300 1300 1320]
 [ 108  108  108 ...  129  129   76]]
writeback 0.0643893350061262


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# vox_refl[...] = HIT/(np.float32(0.1)+HIT+MISS)
# CLASSIFIED_RFM = vox_refl>REF_BLOCK_THRESH
# refl_cache = np.sum(CLASSIFIED_RFM, axis=2)
# HIGHLY_VISIBLE = (np.sum(CLASSIFIED_RFM, axis=2)>wide_threshold)
floodstep()

In [110]:
%matplotlib widget
plt.imshow(np.clip(np.min(FDIST,axis=2),0,255))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Draw Figures
The following calculates the basic RFM in bulk (Faster in Python and good for prototyping).
Then we take a slice out though the glass to make the figure.

We don't discuss it in the paper, due to lack of space, but this also shows several other visualizations good for exploring the RFM


In [ ]:
# # Specialized debug cell
# %matplotlib widget
# vox_refl=HIT/(np.float32(0.1)+HIT+MISS)
# RFM=HIT*w_HIT-MISS*w_MISS
# sud=HIT+MISS
# # del vox_refl
# del sud
# # del vox_refl
# plt.figure(1)
# ax=plt.subplot(221)
# # plt.imshow(np.clip(np.max((HIT),axis=2).T,-3,15))
# plt.imshow(np.clip(np.max((HIT),axis=2).T,-10,10))
# plt.title('Max HIT [0,10]')
# plt.subplot(222,sharex=ax,sharey=ax)
# mean_active = (np.sum(MISS,axis=2,dtype=np.float32)
#               /
#               (np.sum(MISS>0,axis=2)+np.float32(.0001)))[:,:,np.newaxis]
# plt.imshow(np.clip(mean_active[:,:,0].T,-1,10))
# plt.title("Average RFM probes [0,10]")
# plt.subplot(223,sharex=ax,sharey=ax)
# plt.imshow(np.clip(np.max(HIT/(mean_active+0.01),axis=2).T,-1,3))
# plt.title('Normalized Max HIT')

# plt.subplot(224,sharex=ax,sharey=ax)
# plt.imshow(np.clip(np.max(HIT/(mean_active+0.01),axis=2).T,-1,3))
# plt.title('Example RFM Slice Line')
# plt.plot([340,360],[142,142],'r')

# del mean_active


# plt.figure(4)
# plt.title("H Shapes Figure, Option 1")
# # Need to take max of neighboring slices due to aliasing
# plt.imshow(np.clip(np.sum(HIT[:,845:855,:],axis=1).T,0,2),aspect='auto') 
# plt.xlim(320,540)
# # plt.ylim(0,700)
# # plt.imshow(np.clip(np.sum(vox_refl[:,:,:],axis=0).T,0,2))
# plt.xlabel('x [1 unit = 5cm]')
# plt.ylabel('theta [720 units = 180deg]')

# plt.figure(5)
# plt.title("H Shapes Figure,Option 2")
# plt.imshow(np.clip(np.max(RFM[:,142:143,:],axis=1).T,-2,2),aspect='auto',cmap='bwr_r')
# plt.xlabel('x [1 unit = 5cm]')
# plt.ylabel('theta [720 units = 180deg]')
# plt.xlim(295,425)





**Figure captions** \
*Option 1:* Glass panes are easily detectable in the Reflectance Field Map by their distinctive H shape. 
Shown here is a slice through the RFM along a wall made of several panes of glass, producing the repeated H patterns seen here.
The arms of the H are formed by the edges of the glass or its frame which scatter light in many angles,
and the cross bar is caused by the smooth face of the glass reflecting mostly perpendicularly.
Note that despite the wide variation in visibility between different pieces of glass, and the gaps 
in some of the H arms due to occlusions, this wall is reliably detectable at 91deg = 364 units \
[TODO: figure out how to make Figure scale show degrees] \
Unlike intensity or multiecho methods, our's does not require a bright specular peak, or an object behind to be detected.
And unlike Foster et al 2013[Cite], the gaps and secondary detection angles shown here do not confuse our detector.
Even an occluder directly blocking the view of the glass only causes the loss of the blocked part.

*Option 2:*: Similar text to Option 1, but mention Blue is reflective, Red is transparent, and white is areas of the 
reflectance field not probed, due to occlusions or the sensor path. \
Option two is made available to demostrate ability to show alternate color scheme, an alternate target with more regular 
(but less interesting!) pieces of glass, 

In [ ]:
# Classify reflective/transparent per voxel
# Dead, but kept for interest. For some reason this was considered better than the ratio thing at some point
# Now I rememeber. Its because glass has a low visibility ratio, but a high ratio at a cell level
# but it may have few misses, so being > a randomly hit cell s 
# basically a cell with 1 hit and no misses on an unseen location is better than 1 hit and no miss on a heavily seen locaiton 

# def classify_cells_bulk():
#     # Justification: mean_active is an approximation of how many times we should have seen a cell.
#     # It is generated by looking at the mean of HIT+MISS for cells that have ever had either, meaned along the theta axis
#     # Note that if there is a huge variation in the HIT+MISS sum, this could be inaccurate(i.e. if we take different paths 
#     # near the point.)
#     # The score is how many times we saw/expected attempts. We take the max score over angles to represent the most opaque
#     # direction. Scores significantly above 1.0 indicate a bad condition
#     #
#     fullcount=MISS+HIT
#     mean_active = (np.sum(fullcount,axis=2,dtype=np.float32)
#                   /
#                   (np.sum(fullcount>0,axis=2)+np.float32(.0001)))[:,:,np.newaxis]
#     score = np.max(HIT/(mean_active+np.float32(.0001)),axis=2)
#     refl_cache[1:-1,1:-1]=(score>REF_BLOCK_THRESH)[1:-1,1:-1]
#     return score
# classify_cells_bulk()

In [ ]:
%matplotlib widget
plt.imshow(refl_cache.T)
plt.title('Cells that stop rays for reflectance calculations')

In [ ]:
# Hstore=[]
# Mstore=[]

# Show Stabiliation of Reflection Removal:

In [ ]:
# %matplotlib widget

# from tqdm import tqdm
# plt.imshow(refl_cache.T)

# for refitr in range(2):
#     Hstore.append(HIT)
#     Mstore.append(MISS)
    
#     classify_cells_bulk()
    
#     arays=outrays.copy()
#     arays,notref = classify(arays)

#     HIT = np.zeros((W,H,th_bins),'int16')
#     MISS = np.zeros((W,H,th_bins),'int16')
#     ls=0
#     keep=(rays0['r']<30)*(rays0['r']>0)
#     CHUNK_SIZE = 1081

#     for s in tqdm(range(CHUNK_SIZE,arays.shape[1]+1,CHUNK_SIZE)):
#         tmprays=arays[:,ls:s][:,keep[ls:s]]
#         nrtemp=notref[ls:s][keep[ls:s]]
#         accumulate(tmprays, nrtemp)
#         ls=s
        
#     plt.figure()
#     plt.imshow(refl_cache.T)

In [ ]:
# # Visualize the 2d occupancy score
# %matplotlib widget
# score=classify_cells_bulk()
# # plt.imshow(np.clip(score,-5,5).T)
# plt.imshow((score>0.0).T)

# plt.figure()
# plt.imshow(np.clip(np.max(HIT,axis=2),0,10).T)
# print(np.sum(HIT>0))

In [ ]:
# This cell produces a label array rfmcomps, that labels everything that is connected in RFM space, accounting for the wraparound of theta
# It works by labelling connected components, rolling Pi around, relabelling, and then merging the resulting label set

from scipy.ndimage import label
from scipy.ndimage import generate_binary_structure
# help(generate_binary_structure)

%matplotlib widget
vox_refl[...] = HIT/(np.float32(0.1)+HIT+MISS)
structure=generate_binary_structure(3,3)
rfmcomps, count = label(vox_refl>0.3, structure)

roll_vox_refl=np.roll(vox_refl, int(vox_refl.shape[2]//2), axis=2)
rfmcomps2, count = label(roll_vox_refl>0.3, structure)
del roll_vox_refl
offset=np.max(rfmcomps)+1
rfmcomps2=np.roll(rfmcomps2, -int(rfmcomps2.shape[2]//2), axis=2)+offset
rfmcomps2[rfmcomps2==offset]=0
rfmcomps = unify_labels(rfmcomps, rfmcomps2)
del rfmcomps2

In [ ]:
# # Visualize the connected components in RFM space
# import numpy as np
# # to_show=np.random.rand(3,100)
# to_show=np.nonzero(rfmcomps!=rfmcomps[-3,-3,-3])
# per=np.random.permutation(int(np.max(rfmcomps))+1)
# %gui qt

# import mayavi.mlab
# mayavi.mlab.close(all=True)

# mayavi.mlab.points3d(to_show[0], to_show[1], to_show[2],per[rfmcomps[to_show].astype(int)], colormap='spectral', mode = 'points')

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:20]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


In [ ]:
# This cell removes motion from RFM, given a label array for connected components
HIGHLY_VISIBLE = (countvis>wide_threshold)
selection_color = np.max(rfmcomps)+1

%matplotlib widget
plt.imshow(HIGHLY_VISIBLE.T)

# Build an array that has the selection_color on highly visible items, and the old color on everythng else
rfmcomps2 = (rfmcomps > 0)*HIGHLY_VISIBLE[:,:,np.newaxis]*selection_color
rfmcomps2[rfmcomps2==0]=rfmcomps[rfmcomps2==0]

# # Debug: check the selelection array
# to_show=np.nonzero(rfmcomps2!=rfmcomps2[-3,-3,-3])
# per=np.random.permutation(int(np.max(rfmcomps2))+1)
# %gui qt
# import mayavi.mlab
# mayavi.mlab.points3d(to_show[0], to_show[1], to_show[2],per[rfmcomps2[to_show].astype(int)], colormap='spectral',mode= 'point')

# Flood from the seeds
rfmcomps = unify_labels(rfmcomps, rfmcomps2)

In [ ]:
# Visualize after flood fill
%matplotlib widget
print(rfmcomps.shape)
newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])
selected = np.sum(rfmcomps==newselcolor,axis=2)

nevermissed = ((np.sum(HIT,axis=2)>0)&(np.sum(MISS,axis=2)==0))
allkept = 0.5*nevermissed+selected

plt.imshow(np.clip(selected.T,0,15))

In [ ]:
to_show=np.nonzero(rfmcomps!=rfmcomps[-3,-3,-3])
# per=np.random.permutation(int(np.max(rfmcomps2))+1)
%gui qt

newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])

# import mayavi.mlab
# to_show=np.array(to_show).reshape(3,-1)
# print(to_show.shape)
# to_show=to_show[:,to_show[1]>=106]
# to_show=to_show[:,to_show[1]<=121]
# to_show=to_show[:,to_show[0]>=218]
# to_show=to_show[:,to_show[0]<=475]
# to_show=to_show[:,rfmcomps[tuple(to_show)]==newselcolor]

# print(to_show.shape)
# # to_show=to_show[:,to_show[1]>106]
# color = np.sum(vox_refl>0.3,axis=2)[tuple(to_show[:2])]
# # color=score[tuple(to_show[:2])]

# # color=per[rfmcomps[tuple(to_show)].astype(int)]
# mayavi.mlab.close(all=True)
# mayavi.mlab.points3d(to_show[0], to_show[1], to_show[2], color, colormap='spectral', mode= 'point')

In [ ]:
# Show cells that are default kept
%matplotlib widget


my_dpi=100
sc=1.*1.11*.9825
print(sc)
plt.figure(figsize=(nevermissed.shape[0]/my_dpi*sc, nevermissed.shape[1]/my_dpi*sc), dpi=my_dpi)
# nevermissed[:,1::2]=1
plt.imshow(nevermissed.T)
plt.tight_layout()

plt.title("Cells seen but never missed. Keep by default:")

In [ ]:
%matplotlib widget


plt.imshow(allkept.T>0)
# plt.plot(notref,'b,')
# plt.imshow(refl_cache.T>0)


# Now we have a nice view. 
# Time to remove reflections and do it all over again

In [ ]:
# Hstore=[HIT]
# Mstore=[MISS]
refl_cache[...] = (allkept!=0)

In [ ]:
keeprays=(rays0['r']<10)*(rays0['r']>0)

arays,notref = classify(outrays)
doRFM()
#countvis[...] 
#counttrans[...] 
#countratio[...] 
#refl_cache[...]

In [ ]:
from scipy.ndimage import label
from scipy.ndimage import generate_binary_structure

vox_refl=HIT/(np.float32(0.1)+HIT+MISS)
structure=generate_binary_structure(3,3)
rfmcomps, count = label(vox_refl>0.3, structure)

roll_vox_refl=np.roll(vox_refl, int(vox_refl.shape[2]//2), axis=2)
rfmcomps2, count = label(roll_vox_refl>0.3, structure)
del roll_vox_refl
offset=np.max(rfmcomps)+1
rfmcomps2=np.roll(rfmcomps2, -int(rfmcomps2.shape[2]//2), axis=2)+offset
rfmcomps2[rfmcomps2==offset]=0
rfmcomps = unify_labels(rfmcomps, rfmcomps2)
del rfmcomps2

# This cell removes motion from RFM, given a label array for connected components
wide_threshold = 1/12 * HIT.shape[2]
CLASSIFIED_RFM = vox_refl>REF_BLOCK_THRESH
HIGHLY_VISIBLE = (np.sum(CLASSIFIED_RFM, axis=2)>wide_threshold)
selection_color = np.max(rfmcomps)+1

%matplotlib widget
plt.imshow(HIGHLY_VISIBLE.T)
plt.title('Highly Visible Locations')

# Build an array that has the selection_color on highly visible items, and the old color on everythng else
rfmcomps2 = (rfmcomps > 0)*HIGHLY_VISIBLE[:,:,np.newaxis]*selection_color
rfmcomps2[rfmcomps2==0]=rfmcomps[rfmcomps2==0]

# Flood from the seeds
rfmcomps = unify_labels(rfmcomps, rfmcomps2)

# # Debug
# to_show=np.nonzero(rfmcomps!=rfmcomps[-3,-3,-3])
# per=np.random.permutation(int(np.max(rfmcomps2))+1)
# %gui qt
# import mayavi.mlab
# mayavi.mlab.close(all=True)
# mayavi.mlab.points3d(to_show[0], to_show[1], to_show[2],per[rfmcomps[to_show].astype(int)], colormap='spectral', mode= 'point')

In [ ]:
# Show cells that are default kept
%matplotlib widget

nevermissed = ((np.sum(HIT,axis=2)>0)&(np.sum(MISS,axis=2)==0))
my_dpi=100
sc=1.*1.11*.9825
print(sc)
plt.figure(figsize=(nevermissed.shape[0]/my_dpi*sc, nevermissed.shape[1]/my_dpi*sc), dpi=my_dpi)
plt.imshow(nevermissed.T)
plt.tight_layout()

plt.title("Cells seen but never missed. Keep by default:")

# Visualize after flood fill
newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])
selected = np.sum(rfmcomps==newselcolor,axis=2)

plt.figure(figsize=(refl_cache.shape[0]/my_dpi*sc, refl_cache.shape[1]/my_dpi*sc), dpi=my_dpi)
plt.imshow(np.clip(selected.T,0,15))

# Everything We Keep
plt.figure(figsize=(refl_cache.shape[0]/my_dpi*sc, refl_cache.shape[1]/my_dpi*sc), dpi=my_dpi)
allkept = 0.5*nevermissed+selected
plt.imshow(allkept.T)



In [ ]:
# Build Ratio Metric for Occupancy 
countvis = np.sum(CLASSIFIED_RFM, axis=2).astype(np.uint16)
counttrans = np.sum((CLASSIFIED_RFM==0)&(MISS>0), axis=2).astype(np.uint16)
countratio = np.float32(1.0*countvis/(countvis+counttrans+0.0000000001))

markedrfc = (rfmcomps>0)*countratio[:,:,np.newaxis]

In [ ]:
infofile = str(round(rays0['ts'][0]*1e6))+"_"+str(round(rays0['ts'][-1]*1e6))+".info"
print("Saving to:",infofile)
print()

original_stdout = sys.stdout # Save a reference to the original standard output

with open(infofile, 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print("# Transform between map and pixels (opengl flooring convention):")
    print("T_px_f=",T_px_f.tolist())
    print("# Image info")
    print("W,H,th_bins=",(W,H,th_bins))
    print("# Which rays were used:")
    print("start_time=", str(np.min(rays0['ts'])))
    print("end_time=", str(np.max(rays0['ts'])))
    print("# Base dataset:")
    print("rayfile=",str(rayfile))
    sys.stdout = original_stdout 
with open(infofile, 'r') as f:
    print(f.read())
import cv2

def writeim(name, IM):
    # storage format is 16 bit unsigned
    layers=[IM[:,:,layer] for layer in range(IM.shape[2])]
    batches=[np.hstack(layers[off:off+40]) for off in range(0,IM.shape[2],40)]
    rearranged = np.vstack(batches)
    cv2.imwrite(name,rearranged.astype(np.uint16))
writeim("MISS.png", MISS)
writeim("HIT.png", HIT)
cv2.imwrite("refl_cache.png",refl_cache)
cv2.imwrite("allkept.png" , ((allkept>0)*255).astype(np.uint8))
countvis = np.sum(CLASSIFIED_RFM, axis=2).astype(np.uint16)
counttrans = np.sum((CLASSIFIED_RFM==0)&(MISS>0), axis=2).astype(np.uint16)
cv2.imwrite("countvis.png" , countvis)
cv2.imwrite("counttrans.png" , counttrans)
cv2.imwrite("countratio.png" , (countvis*1.0/(counttrans+countvis)*255).astype('uint8'))

# GENERATE FIGURES

### H Figure (RUN BEFORE REMOVALS)

In [ ]:
# ## H figure from North Wall, with background removed
# # Colors show how widely visible the cell is
# # Requires "/bbb3_glass_eval_111716.rays" dataset, with
# #   substart=0
# #   subend=int(2.0e6)
# %matplotlib widget
# plt.imshow(np.clip(np.sum(vox_refl,axis=2),0,15).T, cmap='gray')
# # plt.imshow(allkept.T>0)
# plt.ylim(250,0)
# plt.xlim(200,500)
# # y=[106:121]
# # x=[218:475]
# plt.plot([218,218,475,475,218],[106,121,121,106,106],'c')

# to_show=np.nonzero(rfmcomps!=rfmcomps[-3,-3,-3])
# # per=np.random.permutation(int(np.max(rfmcomps2))+1)
# %gui qt

# newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])

# import mayavi.mlab
# to_show=np.array(to_show).reshape(3,-1)
# print(to_show.shape)
# to_show=to_show[:,to_show[1]>=106]
# to_show=to_show[:,to_show[1]<=121]
# to_show=to_show[:,to_show[0]>=218]
# to_show=to_show[:,to_show[0]<=475]
# to_show=to_show[:,rfmcomps[tuple(to_show)]==newselcolor]

# print(to_show.shape)
# # to_show=to_show[:,to_show[1]>106]
# # color = np.sum(vox_refl>0.3,axis=2)[tuple(to_show[:2])]
# color=(HIT+MISS)[tuple(to_show)]
# # color=score[tuple(to_show[:2])]

# # color=per[rfmcomps[tuple(to_show)].astype(int)]
# mayavi.mlab.close(all=True)
# mayavi.mlab.points3d(to_show[0], to_show[1], to_show[2], color, colormap='spectral', mode= 'point')

In [ ]:
## H figure from 2a, with background removed
# Colors show how widely visible the cell is
# Requires "/bbb3_glass_eval_111716.rays" dataset, with
#   substart=0
#   subend=int(2.0e6)
%matplotlib widget
plt.imshow(np.clip(np.sum(vox_refl,axis=2),0,15).T, cmap='gray')
# plt.imshow(allkept.T>0)
# plt.ylim(250,0)
# plt.xlim(200,500)
# y=[106:121]
# x=[218:475]
plt.plot([218,218,475,475,218],[106,121,121,106,106],'c')

to_show=np.nonzero(HIT!=HIT[-3,-3,-3])
# per=np.random.permutation(int(np.max(rfmcomps2))+1)
%gui qt

# newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])

import mayavi.mlab
to_show=np.array(to_show).reshape(3,-1)
# print(to_show.shape)
to_show=to_show[:,to_show[1]>=1087.5]
to_show=to_show[:,to_show[1]<=1092.5]
# to_show=to_show[:,to_show[0]>=218]
to_show=to_show[:,to_show[0]>464]
to_show=to_show[:,to_show[0]<519]
# to_show=to_show[:,rfmcomps[tuple(to_show)]==newselcolor]

# print(to_show.shape)
# # to_show=to_show[:,to_show[1]>106]
# # color = np.sum(vox_refl>0.3,axis=2)[tuple(to_show[:2])]
color=(HIT+MISS)[tuple(to_show)]
# # color=score[tuple(to_show[:2])]

# # color=per[rfmcomps[tuple(to_show)].astype(int)]
mayavi.mlab.close(all=True)
mayavi.mlab.points3d(to_show[0], to_show[1], to_show[2]/10, color, colormap='spectral', mode= 'point')

In [ ]:
# ## H figure from circle region, with background removed ------------------------------
# # Colors show how widely visible the cell is
# # Requires "/bbb3_glass_eval_111716.rays" dataset, with
# #   substart=int(8e6)
# #   subend=int(11e6)
# %matplotlib widget
# plt.imshow(np.clip(np.sum(vox_refl,axis=2),0,15).T, cmap='gray_r')
# # plt.imshow(allkept.T>0)
# plt.ylim(450,150)
# plt.xlim(350,500)
# # y=[106:121]
# # x=[218:475]
# plt.plot([218,218,475,475,218],[106,121,121,106,106],'c')
# th=np.linspace(0,2*np.pi)
# r=58
# plt.plot(r*np.cos(th)+407,r*np.sin(th)+291,"g")
# r=63
# plt.plot(r*np.cos(th)+407,r*np.sin(th)+291,"g")

# plt.plot(px_ept[:,0][notref==0]-0.5,px_ept[:,1][notref==0]-0.5,'b,')

# plt.figure()
# plt.imshow(counttrans+countvis*10)


# ## 3D half --------------------------------------------------------------------------
# to_show=np.nonzero(rfmcomps!=rfmcomps[-3,-3,-3])

# # per=np.random.permutation(int(np.max(rfmcomps2))+1)
# %gui qt

# newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])

# import mayavi.mlab
# to_show=np.array(to_show).reshape(3,-1)
# print(to_show.shape)
# r=np.sqrt((to_show[0]-407)**2+(to_show[1]-291)**2)
# keep = (r>58) & (r<63)
# to_show=to_show[:,keep]
# to_show=to_show[:,rfmcomps[tuple(to_show)]==newselcolor]

# print(to_show.shape)
# # to_show=to_show[:,to_show[1]>106]

# color = np.clip((np.sum(vox_refl>0.3,axis=2)[tuple(to_show[:2])])/4,0,wide_threshold/4*2)
# color = countratio[tuple(to_show[:2])]

# # color=(HIT+MISS)[tuple(to_show)]
# # color=score[tuple(to_show[:2])]

# # color=per[rfmcomps[tuple(to_show)].astype(int)]
# mayavi.mlab.close(all=True)

# mayavi.mlab.points3d(to_show[0], -to_show[1], to_show[2]/4, color*100, colormap='spectral', mode= 'point')
# f = mayavi.mlab.gcf()
# camera = f.scene.camera

# camera.position=[407,-291,400]
# camera.view_up=[0,1,0]
# camera.focal_point = [407,-291,180]
# camera.view_angle=70

# # Need to roll mouse to rerender
# # 1. Enable legend
# # 2. set ticks to 11
# # 3. set legend actor to tick format %-#6.1g
# # 4. label %reflective
# # 5. set legend actor Vertical Title Separation to 5

# # Try:
# # Whiter background for color contrast
# # Red for walls
# # Blue for glass
# # 

In [ ]:
# ## H figure from circle region, with background removed Fig 1 version ------------------------------
# # Colors show how widely visible the cell is
# # Requires "/bbb3_glass_eval_111716.rays" dataset, with
# #   substart=int(8e6)
# #   subend=int(11e6)
# %matplotlib widget
# plt.imshow(np.clip(np.sum(vox_refl,axis=2),0,15).T, cmap='gray_r')
# # plt.imshow(allkept.T>0)
# xc=234
# yc=304
# plt.ylim(450-291+yc,150-291+yc)
# plt.xlim(350-407+xc,500-407+xc)
# # y=[106:121]
# # x=[218:475]
# plt.plot([218,218,475,475,218],[106,121,121,106,106],'c')

# th=np.linspace(0,2*np.pi)
# r=58
# plt.plot(r*np.cos(th)+xc,r*np.sin(th)+yc,"g")
# r=63
# plt.plot(r*np.cos(th)+xc,r*np.sin(th)+yc,"g")

# plt.plot(px_ept[:,0][notref==0]-0.5,px_ept[:,1][notref==0]-0.5,'b,')

# plt.figure()
# plt.imshow(counttrans+countvis*10)

# ## 3D half --------------------------------------------------------------------------
# to_show=np.nonzero(rfmcomps!=rfmcomps[-3,-3,-3])

# # per=np.random.permutation(int(np.max(rfmcomps2))+1)
# %gui qt

# newselcolor=np.max(rfmcomps[HIGHLY_VISIBLE,:])

# import mayavi.mlab
# to_show=np.array(to_show).reshape(3,-1)
# print(to_show.shape)

# r=np.sqrt((to_show[0]-xc)**2+(to_show[1]-yc)**2)
# keep = (r>58) & (r<63)
# to_show=to_show[:,keep]
# to_show=to_show[:,rfmcomps[tuple(to_show)]==newselcolor]

# print(to_show.shape)
# # to_show=to_show[:,to_show[1]>106]

# color = np.clip((np.sum(vox_refl>0.3,axis=2)[tuple(to_show[:2])])/4,0,wide_threshold/4*2)
# color = countratio[tuple(to_show[:2])]

# # color=(HIT+MISS)[tuple(to_show)]
# # color=score[tuple(to_show[:2])]

# # color=per[rfmcomps[tuple(to_show)].astype(int)]
# mayavi.mlab.close(all=True)

# mayavi.mlab.points3d(to_show[0], -to_show[1], to_show[2]/4, color*100, colormap='spectral', mode= 'point')
# f = mayavi.mlab.gcf()
# camera = f.scene.camera

# camera.position=[xc,-yc,400]
# camera.view_up=[0,1,0]
# camera.focal_point = [xc,-yc,180]
# camera.view_angle=70

# # Need to roll mouse to rerender
# # 1. Enable legend
# # 2. set ticks to 11
# # 3. set legend actor to tick format %-#6.1g
# # 4. label %reflective
# # 5. set legend actor Vertical Title Separation to 5

# # Try:
# # Whiter background for color contrast
# # Red for walls
# # Blue for glass
# # 

# Do GT work

# Stages and Inputs:
1. Base RFM
      1. Require: rays, Optional: is_ref_rays
      2. % visible
      3. countref
      4. vox_refl
      5. [not really done] update flood_cache
      6. update high_viz_seeds
      7. update refl_cache <- threshhold %visible 
2. Motion Removal
      1. Requires: vox_refl, high_viz_seeds
      2. conncomps <- before flood
      3. floodcomps <- after flood
      4. update refl_cache
3. Reflection Removal
      1. Requires: rays, refl_cache
      2. is_ref_rays
4. RFM #2
5. Motion Removal #2



Strategies
[assume all reflection removal immediately followed by RFM]

0. OCC GRID

1. 
RFM

2. 
RFM
MOT

3. 
RFM
MOT
REF

4. 
RFM
REF
REF
REF
REF

5. 
RFM
MOT
REF
MOT

6. 
RFM
REF
MOT
REF
MOT
REF
MOT
REF
MOT
...


      
      

In [ ]:
%matplotlib widget

# load_gt()
# convert to points
# Kd tree for DNN
import cv2
gti = cv2.imread("bbb3_glass_eval_111716-gt.png")

# Detect ablations and fill in 
try: allkept
except NameError: allkept = refl_cache.copy()
try: sc
except NameError: sc = 1.090575
try: selected
except NameError: selected = (refl_cache!=0)


# plt.imshow(gti)
empty = np.min(gti,axis=2)[2::5,2::5].T
plt.imshow((empty + (allkept>0)*500), cmap='gray')

bounds=(np.max(gti,axis=2).T==255)&(np.min(gti,axis=2).T==0)
gtglass = (gti[:,:,0]==255).T&(np.min(gti,axis=2).T==0)
gtmetal = (gti[:,:,2]==255).T&(np.min(gti,axis=2).T==0)
gtdiffuse = (gti[:,:,1]==255).T&(np.min(gti,axis=2).T==0)
gtwall = np.nonzero(bounds)
gtwall[0][:] = gtwall[0]/5
gtwall[1][:] = gtwall[1]/5

plt.figure()
plt.imshow(np.dstack([gtmetal,gtdiffuse,gtglass])*1.0)

assert(np.sum(gtmetal&gtdiffuse)==0)
assert(np.sum(gtmetal&gtglass)==0)
assert(np.sum(gtglass&gtdiffuse)==0)
assert(np.all((gtmetal|gtglass|gtdiffuse)==bounds))



gtrefbarrier = np.zeros_like(refl_cache, bool)
gtrefbarrier[gtwall[0],gtwall[1]] = True

wide_bar = cv2.dilate(((gtrefbarrier!=0)*255).astype('uint8'),np.ones((3,3),np.uint8))
assert(not np.any((wide_bar==0)&(gtrefbarrier!=0)))
print(np.sum(gtwall[0]))
plt.figure()
plt.imshow(gtrefbarrier.T)

Get GT reflection classes

In [ ]:
## Cell loads the stored data from the next cell, that would otherwise take forever
import pickle
with open('dump.pickle','rb') as f:
    ret=pickle.load(f)
closenref=ret['closenref']
realclass=ret['realclass']
inzone=ret['inzone']
lim=len(keeprays)

In [ ]:
lim=len(keeprays)


# VERY EXPENSIVE: touched barrier, got near barrier, went way through barrier
refl_cache[...] = gtrefbarrier
keptrays, realclass = classify(outrays[:,:lim].copy(),pen_depth=0.3)


refl_cache[...] = wide_bar
keptrays, closenref = classify(outrays[:,:lim].copy(),pen_depth=0.0)

refl_cache[...] = (empty==0)
keptrays, inzone = classify(outrays[:,:lim].copy(),pen_depth=0.0)

In [ ]:
%matplotlib widget
plt.close('all')
my_dpi=100
plt.figure(figsize=(refl_cache.shape[0]/my_dpi*sc, refl_cache.shape[1]/my_dpi*sc), dpi=my_dpi)

closeref = (closenref==0)

# something is definitely a reflection if it touches a barrier, goes well through it, and ends not near the barrier

# Base state
# plt.plot(px_ept[:lim][keeprays,0], px_ept[:lim][keeprays,1] , 'w,')

# Reflection
intept = px_ept.astype(int)
inrange = np.clip(px_ept.T, loclip[:2], hiclip[:2]).astype(int)
endfar = (wide_bar[inrange[0],inrange[1]]==0)&keeprays
outer = endfar[:lim]&(realclass==0)
defref = outer&keeprays

plt.plot(px_ept[:lim][defref,0], px_ept[:lim][defref,1] , 'c,')

# Motion
zone=empty
defnotref = inzone&(closeref==0)&keeprays[:lim]
print("defnotref:",np.sum(defnotref))
# plt.imshow(((empty==0).T))
# plt.imshow((gtrefbarrier.T))
import matplotlib as mpl
vimg=np.dstack([gtmetal.T,gtdiffuse.T,gtglass.T])*1.0
# import cv
for i in range(1):
    strel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * 5 + 1, 2 * 5 + 1),
                                       (5, 5))
    vimg=cv2.dilate(vimg.astype(np.float32),strel)
imgplot= plt.imshow(vimg*0,extent=(0,W,H,0))

def fatten(img):
    strel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * 5 + 1, 2 * 5 + 1),
                                       (5, 5))
    vimg=cv2.dilate(img.astype(np.float32),strel)
    return vimg.astype(img.dtype)

# imgplot= plt.imshow(np.clip(selected.T,0,30)*0,cmap='gray',vmin=-10, vmax=30)
transform = mpl.transforms.Affine2D().translate(0.5, 0.5)
imgplot.set_transform(transform + plt.gca().transData)

plt.plot(px_ept[:lim][defnotref,0], px_ept[:lim][defnotref,1] , 'm,')

# Wall core
inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
keepcore = (bounds[inrange5[0],inrange5[1]]!=0)&keeprays
# plt.plot(px_ept[:lim][keepcore,0], px_ept[:lim][keepcore,1] , 'y,')

def easy_read_fig():
    inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
    glasscore = (fatten(gtglass)[inrange5[0],inrange5[1]]!=0)&keeprays&(defref==0)
    plt.plot(px_ept[:lim][glasscore,0], px_ept[:lim][glasscore,1] , 'b,',markersize=1)

    inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
    metalcore = (fatten(gtmetal)[inrange5[0],inrange5[1]]!=0)&keeprays
    plt.plot(px_ept[:lim][metalcore,0], px_ept[:lim][metalcore,1] , 'r,')

    inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
    diffusecore = (fatten(gtdiffuse)[inrange5[0],inrange5[1]]!=0)&keeprays
    plt.plot(px_ept[:lim][diffusecore,0], px_ept[:lim][diffusecore,1] , 'g,',markersize=1)
easy_read_fig()

inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
glasscore = (gtglass[inrange5[0],inrange5[1]]!=0)&keeprays
# plt.plot(px_ept[:lim][glasscore,0], px_ept[:lim][glasscore,1] , 'b,',markersize=1)

inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
metalcore = (gtmetal[inrange5[0],inrange5[1]]!=0)&keeprays
# plt.plot(px_ept[:lim][metalcore,0], px_ept[:lim][metalcore,1] , 'r,')

inrange5 = np.clip(px_ept.T*5, loclip[:2]*5, hiclip[:2]*5).astype(int)
diffusecore = (gtdiffuse[inrange5[0],inrange5[1]]!=0)&keeprays
# plt.plot(px_ept[:lim][diffusecore,0], px_ept[:lim][diffusecore,1] , 'g,',markersize=1)


plt.plot(px_spt[:lim][:,0][rays['pad']==1], px_spt[:lim][:,1][rays['pad']==1] , 'y') #forgot to swap pad and idx
plt.plot(px_spt[:lim][:,0][rays['pad']==0], px_spt[:lim][:,1][rays['pad']==0] , 'y')

plt.gca().legend(['Reflections', 'Motion','Glass','_nolegend_','Diffuse','Sensors Path'])

plt.ylim(1000,700)
plt.xlim(520,710)
assert(np.sum(diffusecore&endfar)==0)
eval_set = np.zeros_like(keeprays)
classes = [defref, defnotref, glasscore, metalcore, diffusecore]
for i in range(len(classes)):
    eval_set |= classes[i]
    for j in range(len(classes)):
        if i==j:
            continue
        print(i,j)
        assert(not np.any(classes[i]&classes[j]))
assert(np.all((glasscore| metalcore|diffusecore)==(keepcore)))

print("Measurable verse:", np.sum(np.vstack(classes)))
print("TP refl:", np.sum(defref&(notref==0)))

if input("Do you want to save the figure (\"yes\" for save)")=='yes':
    plt.savefig('GT Program.png')

In [ ]:
# %matplotlib widget


# plt.imshow(wide_bar.T,extent=(0,W,H,0))
# # # transform = mpl.transforms.Affine2D().translate(0.5, 0.5)
# # # imgplot.set_transform(transform + plt.gca().transData)


# # overlapped=defref&diffusecore

# plt.plot(px_ept[:lim][overlapped,0],px_ept[:lim][overlapped,1],'b.')

# wide_bar[700,864]

In [ ]:
%matplotlib widget
plt.close('all')
plt.figure(figsize=(refl_cache.shape[0]/my_dpi*sc, refl_cache.shape[1]/my_dpi*sc), dpi=my_dpi)

imgplot= plt.imshow(np.clip(selected.T,0,30),cmap='gray_r',vmin=-5, vmax=5, extent=(0,W,H,0))
plt.ylim(1000,700)
plt.xlim(520,710)

plt.savefig('Program Output.png')
# plt.plot(px_ept[:lim][(notref==0),0], px_ept[:lim][(notref==0),1] , 'w,')

In [ ]:
gtgkept=(gtglass[inrange5[0],inrange5[1]]!=0)&(allkept[inrange[0],inrange[1]]!=0)
scratch=np.zeros_like(gtglass)
scratch[inrange5[0,gtgkept],inrange5[1,gtgkept]]=1
print("glass cells kept", np.sum(scratch))
# %matplotlib widget
# plt.imshow(allkept)
gtgkept=(gtglass[inrange5[0],inrange5[1]]!=0)
scratch=np.zeros_like(gtglass)
scratch[inrange5[0,gtgkept],inrange5[1,gtgkept]]=1
print("glass cells in gt", np.sum(scratch))

gtgkept=(gtdiffuse[inrange5[0],inrange5[1]]!=0)&(allkept[inrange[0],inrange[1]]!=0)
scratch=np.zeros_like(gtdiffuse)
scratch[inrange5[0,gtgkept],inrange5[1,gtgkept]]=1
print("diff cells kept", np.sum(scratch))
# %matplotlib widget
# plt.imshow(allkept)
gtgkept=(gtdiffuse[inrange5[0],inrange5[1]]!=0)
scratch=np.zeros_like(gtglass)
scratch[inrange5[0,gtgkept],inrange5[1,gtgkept]]=1
print("diff cells in gt", np.sum(scratch))

gtgkept=(gtmetal[inrange5[0],inrange5[1]]!=0)&(allkept[inrange[0],inrange[1]]!=0)
scratch=np.zeros_like(gtmetal)
scratch[inrange5[0,gtgkept],inrange5[1,gtgkept]]=1
print("metal cells kept", np.sum(scratch))
# %matplotlib widget
# plt.imshow(allkept)
gtgkept=(gtmetal[inrange5[0],inrange5[1]]!=0)
scratch=np.zeros_like(gtglass)
scratch[inrange5[0,gtgkept],inrange5[1,gtgkept]]=1
print("metal cells in gt", np.sum(scratch))



pred_ref = (notref == False)&eval_set
pred_motion = (notref != False)&(allkept[inrange[0],inrange[1]]==0)&eval_set
pred_static = (notref != False)&(allkept[inrange[0],inrange[1]]!=0)&eval_set

print("Real ref:", np.sum((pred_motion|pred_ref)&defref),"(Pred Remove), ", np.sum(pred_static&defref), "(Pred Keep)")
# print("total ref:",np.sum(defref))
print("Real motion:", np.sum((pred_ref|pred_motion)&defnotref),"(Pred Remove), ", np.sum(defnotref&pred_static), "(Pred Keep)")
# print("total real motion:",np.sum(defnotref))
print()



kept_solids = (allkept[inrange[0],inrange[1]]!=0)&keeprays&notref&eval_set
removed_solids = (allkept[inrange[0],inrange[1]]==0)&keeprays&notref&eval_set
removed_refl = (notref==0) & eval_set

import inspect, re

def varname(p):
    for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
        m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
        if m:
            return m.group(1)
total=0
# actual ['defref', 'defnotref', 'glasscore', 'metalcore', 'diffusecore'] -> 
for c,cn in zip(classes[2:],['glass', 'metal', 'diffuse']):
    print('Actual', cn,'&', 
          np.sum(c&pred_static),'(Pred Keep) &',
          np.sum((c&pred_motion)), '(Pred Remove)',
          """\\ \hhline{~*\items{|-}|}""")
    
# print("total attempted:" , np.sum(keepcore)) # remainder is unclear reflection or wall




total_ref = np.sum(defref)
print("Total Reflection", total_ref)
TP_reflection = np.sum((notref==0)*defref)
print("TP Reflection", TP_reflection)
print("Reflection removed", TP_reflection/total_ref)

# total_not_ref = np.sum(defnotref
FP_reflection = np.sum((notref==1)*defnotref)
print("TP Reflection", TP_reflection)
print("Reflection removed", TP_reflection/total_ref)